# Seasonal Model for Hydrological Flow Prediction
 
This notebook implements a seasonal hydrological model for predicting river flow. The model works by computing the average flow for each week of the year from historical training data, creating a seasonal pattern that can be used to forecast future flows. This simple but effective approach captures reegular annual cycles in river discharge.

In [ ]:
from pathlib import Path
import pandas as pd
import xarray as xr
from ombs_senegal.benchmark_model import BenchmarkScores
from ombs_senegal.season import SeasonalityHandler


DATA_PATH = Path("../../data")
OBS_COL = 'débit_insitu'

#### Data loading

In [ ]:

data = pd.read_csv(
    DATA_PATH/'data_cumul.csv', 
    usecols=['time', OBS_COL], 
    index_col='time',
    converters={"time": pd.to_datetime},
    sep=";"
    )
data = data['2012-01-01':]

We will now compute the seasonal variations and show the resutls. First we split data into train and test sets (80-20 split)

In [ ]:
train_mask = data.index < '2019-01-01'
train_data = data[train_mask]
test_data = data[~train_mask]

Now we compute the seasonality from the train set

In [ ]:
seasonality_handler = SeasonalityHandler()
season = seasonality_handler.compute_seasonal_pattern(train_data)

We add all together into a single dataframe

In [ ]:
results = seasonality_handler.append_season(test_data)

We compute the benchmark scores

In [ ]:
benchmark_scores = BenchmarkScores()
scores = benchmark_scores.compute_scores(
    results[["season"]].to_xarray(), results[OBS_COL].to_xarray(), ["rmse", "mae", "nse", "kge"]
    )
scores_df = scores.to_dataframe()

We will finally plot the results

In [ ]:
scores_str = " ".join(f"{c.upper()}: {scores_df[c].values[0]:.2f}" for c in scores_df.columns)
ax = results.plot(title=f"Seasonal Model\n{scores_str}", ylabel="Flow (m³/s)")

Finally we save the data

In [ ]:
season_pred = results[["season"]].to_xarray()
season_pred = xr.merge([
    season_pred,
    scores.to_array("score", name="scores").sel(forecast_horizon="season", drop=True)
     ])
season_pred = season_pred.expand_dims({"model": ["Season"]})
season_pred.to_netcdf(DATA_PATH/"season.nc")